# Tutorial 07 - Nonlinear elliptic problems

In [ ]:
import gmsh
import multiphenicsx.io
import multiphenicsx.mesh

## 1. Mesh generation

In [ ]:
mesh_size = 5e-2

In [ ]:
gmsh.initialize()
gmsh.model.add("nonlinear_elliptic")

In [ ]:
p0 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(1.0, 0.0, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(1.0, 1.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(0.0, 1.0, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
boundary = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])
domain = gmsh.model.geo.addPlaneSurface([boundary])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0, l1, l2, l3], 1)
gmsh.model.addPhysicalGroup(2, [domain], 1)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)